In [2]:
import sys
sys.path.append("/home/ubuntu/caffe/python")
import caffe
import leveldb
import numpy as np
from caffe.proto import caffe_pb2

import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
import pickle

from numpy import genfromtxt

#add cross validation
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

input_x = np.load("train_res152.npz")["arr_0"]
test_x = np.load("test_res152_hidden.npz")["arr_0"]
input_label = genfromtxt('/home/ubuntu/caffe/examples/images/Files/train.csv', delimiter=',')
input_label = input_label[1:7001,1].reshape(-1)

print (input_x.shape)
print (test_x.shape)

(7000, 2048)
(2000, 2048)


In [3]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


#for max_iter in [100,500,1000,3000]:
#for C_pram in [0.0005,0.002,0.005,0.007,0.1,0.5,1]: #best 0.001,0.01 #best of best 0.001
#for s in ['lbfgs']: #newton-cg tested [0.80840456  0.82571429  0.81357143  0.8084346   0.80601288] #sag too slow
            #for multi_class in ['ovr','multinomial']:
#for C_pram in [0.001]:
clf = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.001, fit_intercept=True,
                       intercept_scaling=1, class_weight=None, random_state=None, 
                       solver='lbfgs', max_iter=3000, multi_class='multinomial',
                       verbose=0, warm_start=False, n_jobs=-1)

# clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
#                                  max_depth=1, random_state=0)
#clf = AdaBoostClassifier(n_estimators=100)
# scores = cross_val_score(clf, input_x, input_label, cv=5)
# print(scores)
# print(scores.mean)

#tol 0.00001 [ 0.81980057  0.82357143  0.81285714  0.81272337  0.80100215]

clf.fit(input_x, input_label)
print ("Predictin the labels")
y_pred = clf.predict(test_x)
filename = "predict_res152_hidden.csv"
f = open(filename, "w")
f.write('Id,Prediction\n')

if ((len(y_pred))<1000):
   zeros = np.zeros(2000)
   y_pred = np.append(y_pred, zeros).reshape(-1)

for i in range(0,len(y_pred)):
   d = '{0},{1}\n'.format(i+1,int(y_pred[i]))
   f.write(d)
print ("Writing Done!")


Predictin the labels
Writing Done!


In [55]:
#Corss Validation
# clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=900,probability=False)
# scores = cross_val_score(clf, input_x, input_label, cv=5)
# print (scores)
# clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=800,probability=False)
# scores = cross_val_score(clf, input_x, input_label, cv=5)
# print (scores)
# clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=700,probability=False)
# scores = cross_val_score(clf, input_x, input_label, cv=5)
# print (scores)
# clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=600,probability=False)
# scores = cross_val_score(clf, input_x, input_label, cv=5)
# print (scores)
# clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=400,probability=False)
# scores = cross_val_score(clf, input_x, input_label, cv=5)
# print (scores)

import random
#set random seed
random.seed(42)
#set cross valid test size
test_size = 0.25
#get a test size list
sample  = random.sample(range(len(input_x)/4),int(len(input_x)/4*test_size))
test_sample = np.asarray(sample)
#get the other part of index as train
mask = np.ones(len(input_x)/4, dtype=bool)
mask[sample] = False
universal = np.arange(len(input_x)/4)
train_sample = universal[mask]

#Extent to whole test_set & test_label
#test_sample_u is the index matrix of test data
test_sample_u = np.append(test_sample,test_sample+len(input_x)/4) #90 degree
test_sample_u = np.append(test_sample_u,test_sample+2*len(input_x)/4) #180 degree
test_sample_u = np.append(test_sample_u,test_sample+3*len(input_x)/4) #270 degree

#Extent to whole train_set & train_label
#train_sample_u is the index matrix of test data
train_sample_u = np.append(train_sample,train_sample+len(input_x)/4)
train_sample_u = np.append(train_sample_u,train_sample+2*len(input_x)/4) #180 degree
train_sample_u = np.append(train_sample_u,train_sample+3*len(input_x)/4) #270 degree


#X_train, X_test, y_train, y_test = train_test_split(input_x, input_label, test_size=0.1, random_state=42)
X_train = input_x[train_sample_u]
X_test = input_x[test_sample_u]
y_train = input_label[train_sample_u]
y_test = input_label[test_sample_u]

print("Split the train data & test data")


Split the train data & test data


In [56]:
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=2000,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=1500,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=1000,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=600,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


('training accuracy is', 1.0)
('validation accuracy is', 0.76671428571428568)


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


('training accuracy is', 1.0)
('validation accuracy is', 0.7658571428571429)


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


('training accuracy is', 0.99971428571428567)
('validation accuracy is', 0.76371428571428568)
('training accuracy is', 0.97799999999999998)
('validation accuracy is', 0.75328571428571434)


In [57]:
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=-1,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=800.0,decision_function_shape='ovr',max_iter=-1,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=3000,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=4000,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))

('training accuracy is', 1.0)
('validation accuracy is', 0.76600000000000001)
('training accuracy is', 1.0)
('validation accuracy is', 0.76600000000000001)


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


('training accuracy is', 1.0)
('validation accuracy is', 0.76700000000000002)


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


('training accuracy is', 1.0)
('validation accuracy is', 0.76600000000000001)


In [ ]:
clf = SVC(C=1.0,decision_function_shape='ovr',max_iter=-1,probability=False)
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))

In [ ]:
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=3500,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=5000,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))

In [ ]:
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=6000,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))
clf = SVC(C=100.0,decision_function_shape='ovr',max_iter=7000,probability=False)
clf.fit(X_train, y_train)
print('training accuracy is', clf.score(X_train,y_train))
print('validation accuracy is', clf.score(X_test,y_test))